In [ ]:
#Primeiro Passo

import warnings
import pandas as pd
import os 
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.core.common import SettingWithCopyWarning

In [ ]:
#Primeiro Passo

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

In [ ]:
#Segundo passo 

diretorioCombustivel = 'C:\\Temp\\Combustiveis\\gasolina_Etanol_Diesel_GNv\\'

dfCombustivel = pd.Dataframe()

for arquivoCombustivel in os.listdir(diretorioCombustivel):
    pathCombustivel = diretorioCOmbustivel + arquivoCombustivel
    print(pathCombustivel)

    df = pd.read.csv(pathCombustivel
                     , sep =';'
                     , header = 0
                     , usecols = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
                     , names = ['Região', 'UF', 'NomMunicipio', 'NomRevenda', 'NumCNPJRevenda', 'NomLogradouro', 'NrEndereço',
                                'NomComplemento', 'NomBairro', 'NumCep', 'DscProduto','DtColeta', 'VlrVenda', 'VlrCompleto',
                                'UnMedida', 'DscBandeira'
                                ]
                     , dtype = {'Região':'str'
                     , 'UF': 'str'
                     , 'NomMunicipio': 'str'
                     , 'NomRevenda': 'str'
                     , 'NumCNPJRevenda': 'str'
                     , 'NomLogradouro': 'str'
                     , 'NrEndereço': 'str'
                     , 'NomComplemento': 'str'
                     , 'NomBairro': 'str', 
                     , 'NumCEP': 'str'
                     , 'DscProduto': 'str'
                     , 'DtColeta': 'str'
                     , 'VlrVenda': 'str'
                     , 'VlrCompleto': 'str'
                     , 'UnMedida': 'str'
                     , 'DscBandeira': 'str'
                     }
                     , parse_dates = [11]
                     , dayfirst = True
                     , engine = 'python')
    
    dfCombustivel = dfCombustivel.append(df)

In [ ]:
#Terceiro Passo 

#Tratamento de dados nulos 
dfCombustivel['NomComplemento'].fillna('NI', inplace=True)
dfCombustivel['NomLogradouro'].fillna('NI',inplace=True)
dfCombustivel['NomBairro'].fillna('NI', inplace=True)
dfCombustivel['VlrCompra'].fillna('NI', inplace=True)

#Criação de Colunas de tempo - Ano, Mes, Anomes
dfCombustivel['Ano'] = dfCombustivel['DtColeta'].dt.strftime('%Y')
dfCombustivel['Mes'] = dfCombustivel['DtColeta'].dt.strftime('%m')
dfCombustivel['AnoMes'] = dfCombustivel['DtColeta'].dt.strftime('%Y%m')

#Criação de coluna endereço completo
dfCombustivel['NomEnderecoCompleto'] = dfCombustivel['Logradouro'] \
                                        + ', NR: ' \
                                        + dfCombustivel['NrEndereço'] \
                                        + ', Complemento: ' \
                                        + dfCombustivel['NomComplemento'] \
                                        + ', Bairro: ' \
                                        + dfCombustivel['NomBairro']

dfCombustivel['NomEnderecoCompleto'].fillna('NI', inplace=True)

#Conversão de tipo de dados
dfCombustivel['VlrCompra'] = dfCombustivel['VlrCompra'].str.replace(',','.').astype(float)
dfCombustivel['VlrVenda'] = dfCombustivel['VlrVenda'].str.replace(',','.').astype(float)

In [ ]:
#Quarto passo

dfCombustivel.info()

In [ ]:
#Quinto passo 

#Aplicação de filtro para retornar apenas os dados que o cliente atua

resultEstadosAtuantes = dfCombustivel[((dfCombustivel['UF']=='CE') & (dfCombustivel['NomMunicipio']=='JUAZEIRO DO NORTE')) |
                                      ((dfCombustivel['UF']=='MA') & (dfCombustivel['NomMunicipio']=='IMPERATRIZ')) |
                                      ((dfcombustivel['UF']=='RJ') & (dfCombustivel['NomMunicipio']=='SAO GONCALO')) |
                                      ((dfCombsutivel['UF']=='SP') & (dfCombustivel['NomMunicipio']=='BARUERI'))
                                    ].groupby(['Regiao'
                                              , 'UF'
                                              , 'NomMunicipio'
                                              , 'Nombairro'
                                              , 'NomRevenda'
                                              , 'NomEnderecoCompleto'
                                              , 'DscProduto'
                                              , 'AnoMes'
                                              , 'Ano'
                                              , 'Mes'
                                              ], as_index=False).agg(VlrMedio=('VlrVenda', 'mean'))

In [ ]:
#sexto passo 

#Transformação de linhas em colunas - colunas - anomes
#criando uma coluna para cada anomes

resultRelatorioFinal = resultEstadosAtuantes.pivot(index=['Regiao'
                                                        , 'UF'
                                                        , 'NomMunicipio'
                                                        , 'NomBairro'
                                                        , 'NomRevenda'
                                                        , 'NomEnderecoCompleto'
                                                        , 'DscProduto'
                                                        ], columns='AnoMes', values='VlrMedio').fillna(0).reset_index()
resultRelatorioFinal

In [ ]:
#Sétimo passo
#Geração de um arquivo pra cada filial 

for lista_uf in resultRelatorioFinal['UF'].unique():
    print("Arquivo Estado do " + lista_uf + 'gerado com sucesso!')

    resultRelatorioFinal[resultRelatorioFInal['UF']==lista_uf].to_csv('C:\\Arquivo\\lista_{0}.csv'
                                                                        .format(lista_uf),sep= ';', index=False)

In [ ]:
#Oitavo Passo
#Representando a variação mensão do municipio em gráfico

reportGrafico = dfCombustivel[((dfCombustivel['UF']=='CE') & (dfCombustivel['NomMunicipio']=='JUAZEIRO DO NORTE'))
                                      ].groupby(['Regiao'
                                               , 'UF'
                                               , 'NomDomicilio'
                                               , 'DscProduto'
                                               , 'AnoMes'
                                               , 'Ano'
                                               , 'Mes'
                                               ], as_index=False).agg(VlrMedio=('VlrVenda', 'mean')).sort_values(by='AnoMes')

In [ ]:
#Nono passo

import smtplib
from email.mine.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

for lista_uf in resultRelatorioFinal['UF'].unique():
    print('Arquivo Estado do ' + lista_uf + ' gerado com sucesso!')

    resultRelatorioFinal[resultRelatorioFinal['UF']==lista_uf].to_csv('C:\\Arquivo\\lista_{0}.csv'
                                                                        .format(lista_uf),sep=';', index=False)
    
    fromaddr = 'seuemail@seuemail.com.br'
    toaddr   = 'destinatario@email.com.br'

    msg = MIMEMultipart()
    msg['From'] = fromaddr
    msg['To']   = toaddr
    msg['Subject'] = "Arquivo de variação dos preços dos combustiveis do estado {0}".format(lista_uf)
    body = 'Olá tudo bem? \nSegue em anexo o arquivo de Variação dos preços dos combustiveis'.format(lista_uf)
    msg.attach(MIMEText(boidy, 'plain'))
    filename = 'lista_[0].csv'.format(lista_uf)
    attachment = open("C:\\Arquivo\lita_[0}.csv".format(lista_uf), 'rb')
    p = MIMEBase('application', 'octet-stream')
    p.set_payload((attachment).read())
    encoders.encode_base64(p)
    p.add_header('Content-Disposition', 'attachment; filename = %s' % filename)
    msg.attach(p)
    s = smtplib.SMTP('smtp@gmail.com', 587)
    s.starttls()
    s.login(fromaddr, 'xxxxxxxxxxxxxxxxxxxxx')
    text = msg.as_string()
    s.sendmail(fromaddr, toaddr, text)
    s.quit()